# Query knowledge graphs with KGTK Kypher

This tutorial gives an overview of the core features of the KGTK Kypher query language.  

For additional details refer to the reading list at the end. 

## Step 0: Install KGTK

Only run the following cell if KGTK is not installed.
 For example, if running in [Google Colab](https://colab.research.google.com/)

In [1]:
#!pip install kgtk

### Notebook data setup

In [2]:
import os
import os.path

from kgtk.configure_kgtk_notebooks import ConfigureKGTK

In [3]:
# Parameters

# Folders on local machine where to create the output and temporary files:
input_path = None
output_path = "/tmp/projects"
project_name = "tutorial-kypher"

In [4]:
# These are all the KG files that we use in this notebook:
additional_files = {
    "small_graph": "small-graph.tsv",
    "small_films": "small-films.tsv",
    "small_props": "small-props.tsv",
    "small_quals": "small-quals.tsv",
}

big_files = [
    "all",
    "label",
    "pagerank_undirected",
]

ck = ConfigureKGTK(big_files)
ck.configure_kgtk(input_graph_path=input_path, 
                  output_path=output_path, 
                  project_name=project_name,
                  additional_files=additional_files)

User home: /root
Current dir: /content
KGTK dir: /
Use-cases dir: //use-cases
--2022-10-04 22:40:24--  https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold/all.tsv.gz
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz [following]
--2022-10-04 22:40:25--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/all.tsv.gz [following]
--2022-10-04 22:40:25--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/all.tsv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.19

In [5]:
ck.print_env_variables()

KGTK_LABEL_FILE: /root/isi-kgtk-tutorial/tutorial-kypher_input/labels.en.tsv.gz
kgtk: kgtk
STORE: /tmp/projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
USE_CASES_DIR: //use-cases
KGTK_GRAPH_CACHE: /tmp/projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
kypher: kgtk query --graph-cache /tmp/projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
EXAMPLES_DIR: //examples
GRAPH: /root/isi-kgtk-tutorial/tutorial-kypher_input
TEMP: /tmp/projects/tutorial-kypher/temp.tutorial-kypher
OUT: /tmp/projects/tutorial-kypher
all: /root/isi-kgtk-tutorial/tutorial-kypher_input/all.tsv.gz
label: /root/isi-kgtk-tutorial/tutorial-kypher_input/labels.en.tsv.gz
pagerank_undirected: /root/isi-kgtk-tutorial/tutorial-kypher_input/metadata.pagerank.undirected.tsv.gz
small_graph: /root/isi-kgtk-tutorial/tutorial-kypher_input/small-graph.tsv
small_films: /root/isi-kgtk-tutorial/tutorial-kypher_input/small-films.tsv
small_props: /root/isi-kgtk-tut

## Usage of `query` command
```
usage: kgtk query [-h] [-i INPUT_FILE [INPUT_FILE ...]] [--as NAME]
                  [--comment COMMENT] [--query QUERY] [--match PATTERN]
                  [--where CLAUSE] [--opt PATTERN] [--with CLAUSE]
                  [--where: CLAUSE] [--return CLAUSE] [--order-by CLAUSE]
                  [--skip CLAUSE] [--limit CLAUSE] [--para NAME=VAL]
                  [--spara NAME=VAL] [--lqpara NAME=VAL] [--no-header]
                  [--force] [--index MODE [MODE ...]] [--idx SPEC [SPEC ...]]
                  [--explain [MODE]] [--graph-cache GRAPH_CACHE_FILE]
                  [--show-cache] [--import MODULE_LIST] [-o OUTPUT]

Query one or more KGTK files with Kypher.
IMPORTANT: input can come from stdin but chaining queries is not yet supported.

Input options:
  -i INPUT_FILE [INPUT_FILE ...], --input-files INPUT_FILE [INPUT_FILE ...]
                        One or more input files to query, maybe compressed
                        (May be omitted or '-' for stdin.)
  --as NAME             alias name to be used for preceding input
  --comment COMMENT     comment string to store for the preceding input
                        (displayed by --show-cache)

Query options:
  --query QUERY         complete Kypher query combining all clauses, if
                        supplied, all other specialized clause arguments will
                        be ignored
  --match PATTERN       MATCH pattern of a Kypher query, defaults to universal
                        node pattern `()'
  --where CLAUSE        WHERE clause to a preceding --match, --opt or --with
                        clause
  --opt PATTERN, --optional PATTERN
                        OPTIONAL MATCH pattern(s) of a Kypher query (zero or
                        more)
  --with CLAUSE         WITH clause of a Kypher query (only 'WITH * ...' is
                        currently supported)
  --where: CLAUSE       final global WHERE clause, shorthand for 'WITH * WHERE
                        ...'
  --return CLAUSE       RETURN clause of a Kypher query (defaults to *)
  --order-by CLAUSE     ORDER BY clause of a Kypher query
  --skip CLAUSE         SKIP clause of a Kypher query
  --limit CLAUSE        LIMIT clause of a Kypher query
  --para NAME=VAL       zero or more named value parameters to be passed to
                        the query
  --spara NAME=VAL      zero or more named string parameters to be passed to
                        the query
  --lqpara NAME=VAL     zero or more named LQ-string parameters to be passed
                        to the query

Output options:
  --no-header           do not generate a header row with column names
  -o OUTPUT, --out OUTPUT
                        output file to write to, if `-' (the default) output
                        goes to stdout. Files with extensions .gz, .bz2 or .xz
                        will be appropriately compressed.

Control options:
  --force               force problematic queries to run against advice
  --index MODE [MODE ...], --index-mode MODE [MODE ...]
                        default index creation MODE for all inputs (default:
                        auto); can be overridden with --idx for specific
                        inputs
  --idx SPEC [SPEC ...], --input-index SPEC [SPEC ...]
                        create index(es) according to SPEC for the preceding
                        input only
  --explain [MODE]      explain the query execution and indexing plan
                        according to MODE (plan, full, expert, default: plan).
                        This will not actually run or create anything.
  --graph-cache GRAPH_CACHE_FILE
                        database cache where graphs will be imported before
                        they are queried (defaults to per-user temporary file)
  --show-cache          describe the current content of the graph cache and
                        exit (does not actually run a query or import data)
  --import MODULE_LIST  Python modules needed to define user extensions to
                        built-in functions
  -h, --help            show this help message and exit
```

## Selecting edges with the `--match` clause

- KGTK `query` either takes a full Kypher `--query` or individual Kypher clauses such as `--match`,
  `--return`, etc.
- individual clauses are automatically assembled into the proper order, easier in Unix shell environment

**Example**
- simple query on a single input graph with an anonymous edge pattern
- for convenience we use shell variable `$GRAPH` to point to small data file
- **IMPORTANT**: use quotes around the match pattern to protect from the shell
- this matches all edges and returns the whole file

In [6]:
!echo $small_graph

/root/isi-kgtk-tutorial/tutorial-kypher_input/small-graph.tsv


In [7]:
!kgtk query -i $small_graph --match '()-[]->()'

id	node1	label	node2
e1	m_shriver	spouse	a_schwarzenegger
e2	a_schwarzenegger	coactor	l_hamilton
e3	g_rydstrom	colleague	g_borders
e4	g_rydstrom	colleague	a_schwarzenegger
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de
e6	l_hamilton	name	'Linda Hamilton'@en
e7	g_rydstrom	name	'Gary R. Rydstrom'@en
e8	g_borders	name	'Gloria S. Borders'@en
e9	m_shriver	name	'Maria Shriver'@en


- match pattern starts with anonymous node connecting via an
  anonymous relation to another anonymous node
- matched against the four core columns specifying an edge in each line of the KGTK
  input file
- from-node is matched against `node1`, relation is
  matched against `id` and `label` (more on that distinction later), and
  the to-node is matched against `node2`
- for each KGTK line matching the pattern, output is generated according to the `--return` clause
- default for `--return` is `*` which means all columns of a matching line will be output

**Equivalent query:**
- singular anonymous node pattern will be completed to a full edge

In [8]:
!kgtk query -i $small_graph --match '()'

id	node1	label	node2
e1	m_shriver	spouse	a_schwarzenegger
e2	a_schwarzenegger	coactor	l_hamilton
e3	g_rydstrom	colleague	g_borders
e4	g_rydstrom	colleague	a_schwarzenegger
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de
e6	l_hamilton	name	'Linda Hamilton'@en
e7	g_rydstrom	name	'Gary R. Rydstrom'@en
e8	g_borders	name	'Gloria S. Borders'@en
e9	m_shriver	name	'Maria Shriver'@en


**Equivalent query:**
- that pattern is also the default for `--match`

In [9]:
!kgtk query -i $small_graph

id	node1	label	node2
e1	m_shriver	spouse	a_schwarzenegger
e2	a_schwarzenegger	coactor	l_hamilton
e3	g_rydstrom	colleague	g_borders
e4	g_rydstrom	colleague	a_schwarzenegger
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de
e6	l_hamilton	name	'Linda Hamilton'@en
e7	g_rydstrom	name	'Gary R. Rydstrom'@en
e8	g_borders	name	'Gloria S. Borders'@en
e9	m_shriver	name	'Maria Shriver'@en


**Equivalent command:** `kgtk cat`

In [10]:
!kgtk cat -i $small_graph

id	node1	label	node2
e1	m_shriver	spouse	a_schwarzenegger
e2	a_schwarzenegger	coactor	l_hamilton
e3	g_rydstrom	colleague	g_borders
e4	g_rydstrom	colleague	a_schwarzenegger
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de
e6	l_hamilton	name	'Linda Hamilton'@en
e7	g_rydstrom	name	'Gary R. Rydstrom'@en
e8	g_borders	name	'Gloria S. Borders'@en
e9	m_shriver	name	'Maria Shriver'@en


### Restricting output
- use `--limit` and/or `--skip` to control output volume,
  especially to test queries on larger data

In [11]:
!kgtk query -i $small_graph --limit 3

id	node1	label	node2
e1	m_shriver	spouse	a_schwarzenegger
e2	a_schwarzenegger	coactor	l_hamilton
e3	g_rydstrom	colleague	g_borders


In [12]:
!kgtk query -i $small_graph --skip 2 --limit 3

id	node1	label	node2
e3	g_rydstrom	colleague	g_borders
e4	g_rydstrom	colleague	a_schwarzenegger
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de


In [13]:
!kgtk query -i $small_graph / head -n 3

id	node1	label	node2
e1	m_shriver	spouse	a_schwarzenegger
e2	a_schwarzenegger	coactor	l_hamilton
e3	g_rydstrom	colleague	g_borders


### Restricting edges
- patterns can be made more specific by restricting some of the elements of an edge
- for example, filter for all edges that start with `m_shriver`

In [14]:
!kgtk query -i $small_graph --match '(:m_shriver)-[]->()'

id	node1	label	node2
e1	m_shriver	spouse	a_schwarzenegger
e9	m_shriver	name	'Maria Shriver'@en


- **Kypher vs. Cypher**: in Cypher the restriction `m_shriver` would be interpreted as a node *type* in a property graph, in KGTK Kypher
it is interpreted as the ID of a particular node

**Example**
- filter on the relation of an edge
- select all edges with label `name` using the same colon-restriction syntax we used for nodes

In [15]:
!kgtk query -i $small_graph --match '()-[:name]->()'

id	node1	label	node2
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de
e6	l_hamilton	name	'Linda Hamilton'@en
e7	g_rydstrom	name	'Gary R. Rydstrom'@en
e8	g_borders	name	'Gloria S. Borders'@en
e9	m_shriver	name	'Maria Shriver'@en


- for relations, the interpretation of restrictions on the label of an
edge (as opposed to its `id`) is more in line with standard Cypher.

**Example**
- node and relation restrictions can be combined
- for example, here we select all `name` edges starting from node `a_schwarzenegger`

In [16]:
!kgtk query -i $small_graph --match '(:a_schwarzenegger)-[:name]->()'

id	node1	label	node2
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de


## Filtering with the `--where` clause

- `--where` holds a possibly complex Boolean expression that gets evaluated as additional edge filter
- only edges for which it evaluates to true will be returned
- `--where` can be used to express more complex conditions and computations that cannot be expressed in `--match`
- to connect to values selected by `--match` we need pattern *variables*
- variables are specified with simple names in the node or relationship part of a pattern
- below we use `p` as the variable for the starting node of the edge pattern
- in the `--where` clause we restrict which values are allowed for `p`
- this query is equivalent to the one above where we restricted the starting node directly in the match pattern:

In [17]:
!kgtk query -i $small_graph \
     --match '(p)-[:name]->()' \
     --where 'p = "m_shriver"'

id	node1	label	node2
e9	m_shriver	name	'Maria Shriver'@en


**Equivalent query:**
- specifies the starting node restriction twice which is ok but redundant

In [18]:
!kgtk query -i $small_graph \
     --match '(p:m_shriver)-[:name]->()' \
     --where 'p = "m_shriver"'

id	node1	label	node2
e9	m_shriver	name	'Maria Shriver'@en


- **IMPORTANT**: constants such as `m_shriver` need to be quoted when used in `--where`, similar to SQL
- this needs extra care so quotes will not be consumed by the Unix shell

**Example**
- use a regular expression to filter on the names attached to nodes
- the `=~` operator matches a value against a regular expression
- **Kypher vs. Cypher**: Kypher use Python regexp syntax, Cypher uses Java regexps
- select all `name` edges that lead to a name that contains a double letter:

In [19]:
!kgtk query -i $small_graph \
     --match '(p)-[:name]->(n)' \
     --where 'n =~ ".*(.)\\1.*"'

id	node1	label	node2
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de


**Example**
- filter based on a list of values (a form of disjunction)
- any edge where `p` is equal to one of the listed values will be returned
- **Kypher vs. Cypher**: Kypher only allows lists of literals such as strings
  or numbers, Cypher also allows variables and expressions

In [20]:
!kgtk query -i $small_graph \
     --match '(p)-[:name]->(n)' \
     --where 'p IN ["g_borders", "g_rydstrom"]'

id	node1	label	node2
e7	g_rydstrom	name	'Gary R. Rydstrom'@en
e8	g_borders	name	'Gloria S. Borders'@en


**Example**
- filter based on a comparison operator and a computation using built-in functions
- **IMPORTANT**: all columns in a KGTK file are treated as text (even if they contain numbers)
- the expression below filters for names that start with the letter `L` or later
- note that quotes of KGTK string literals are part of their value and need to be accounted for
- for this we use the built-in function `substr` to extract the first letter of each name

In [21]:
!kgtk query -i $small_graph \
     --match '(p)-[:name]->(n)' \
     --where "substr(n,2,1) >= 'L'"

id	node1	label	node2
e6	l_hamilton	name	'Linda Hamilton'@en
e9	m_shriver	name	'Maria Shriver'@en


## Sorting results with the `--order-by` clause

- use `--order-by` to sort results just like in Cypher and SQL
- this query sorts matched results by names in ascending order:

In [22]:
!kgtk query -i $small_graph \
     --match '(p)-[:name]->(n)' \
     --where "upper(substr(n,2,1)) >= 'L'" \
     --order-by n

id	node1	label	node2
e6	l_hamilton	name	'Linda Hamilton'@en
e9	m_shriver	name	'Maria Shriver'@en


- order by an expression value, this time in descending order:

In [23]:
!kgtk query -i $small_graph \
     --match '(p)-[:name]->(n)' \
     --where "substr(n,2,1) >= 'L'" \
     --order-by "substr(n,2,1) desc"

id	node1	label	node2
e9	m_shriver	name	'Maria Shriver'@en
e6	l_hamilton	name	'Linda Hamilton'@en


## Controlling results with the `--return` clause

- use `--return` to control which columns are output, in which order, with what headers
- by default `*` is used which means all columns of matching edge(s) are output
- use KGTK and SQLite built-in functions to transform output values or compute additional ones
- use `distict` to eliminate duplicates
- use aggregation function to get counts, max, average, etc.

**Example**
- select only the `node1` and `node2` columns by referencing pattern variables `p` and `n`
- note, that the result generated here is not valid KGTK (missing `id` and `label`)

In [24]:
!kgtk query -i $small_graph \
     --match '(p)-[:name]->(n)' \
     --where 'n =~ ".*(.)\\1.*"' \
     --return 'p, n'

node1	node2
a_schwarzenegger	'Arnold Schwarzenegger'@de


**Example**
- return all columns but switching their order
- **Kypher vs. Cypher**: Kypher relation variables such as `r` below
  get bound to edge IDs; other components of an edge such as its `label`
  can then be access via property syntax such as `r.label`
- this query now produces valid KGTK

In [25]:
!kgtk query -i $small_graph \
     --match '(p)-[r:name]->(n)' \
     --where 'n =~ ".*(.)\\1.*"' \
     --return 'p, n, r, r.label'

node1	node2	id	label
a_schwarzenegger	'Arnold Schwarzenegger'@de	e5	name


**Example**
- summarize data via `distinct` to eliminate duplicates

In [26]:
!kgtk query -i $small_graph \
     --match '(p)-[r]->(n)' \
     --return 'distinct r.label' \
     --order-by r.label

label
coactor
colleague
name
spouse


**Example**
- transform data with built-in functions, e.g., change value to lower case

In [27]:
!kgtk query -i $small_graph \
     --match '(p)-[r:name]->(n)' \
     --where 'n =~ ".*(.)\\1.*"' \
     --return 'p, r.label, lower(n), r'

node1	label	lower(graph_1_c1."node2")	id
a_schwarzenegger	name	'arnold schwarzenegger'@de	e5


**Example**
- specify proper column headers for output values, e.g., designate the transformed value as `node2`

In [28]:
!kgtk query -i $small_graph \
     --match '(p)-[r:name]->(n)' \
     --where 'n =~ ".*(.)\\1.*"' \
     --return 'p, r.label, lower(n) as node2, r'

node1	label	node2	id
a_schwarzenegger	name	'arnold schwarzenegger'@de	e5


**Example**
- use KGTK built-in functions to manipulate KGTK values
- here we change a symbol to a string (which is a legal `node1` in KGTK data model)

In [29]:
!kgtk query -i $small_graph \
     --match '(p)-[r:name]->(n)' \
     --where 'n =~ ".*(.)\\1.*"' \
     --return 'kgtk_stringify(p) as node1, r.label, n, r'

node1	label	node2	id
"a_schwarzenegger"	name	'Arnold Schwarzenegger'@de	e5


**Example**
- access components of structured literals with property syntax (treat them as *virtual properties*)
- but this is just syntactic sugar for calling the KGTK function
- extra output column is named with column path syntax (`lang` of `node2`)
- special characters need to be backtick-quoted

In [30]:
!kgtk query -i $small_graph \
     --match '(p)-[r:name]->(n)' \
     --where 'kgtk_lqstring(n) and n.kgtk_lqstring_lang != "en"' \
     --return 'r, p, r.label, lower(n) as node2, n.kgtk_lqstring_lang as `node2;lang`'

id	node1	label	node2	node2;lang
e5	a_schwarzenegger	name	'arnold schwarzenegger'@de	de


## Querying connected edges through graph patterns

- in KGs  we will often want to combine multiple edges into a query
- connection could be within a graph or across graphs or both
- in database parlance this is generally called a *join*
- in Kypher (or Cypher) we can express such queries very concisely using graph patterns
- this ASCII pattern language is probably the single-most useful aspect of K/Cypher
- for example, find colleagues and their names:

In [31]:
!kgtk query -i $small_graph \
     --match '(na)<-[:name]-(a)-[r:colleague]->(b)-[:name]->(nb)' \
     --return 'r, na as node1, r.label, nb as node2'

id	node1	label	node2
e3	'Gary R. Rydstrom'@en	colleague	'Gloria S. Borders'@en
e4	'Gary R. Rydstrom'@en	colleague	'Arnold Schwarzenegger'@de


**Example**
- combine path patterns with `--where` expression for more elaborate filtering
- below we select starting edges where at least one of the nodes has a German name

In [32]:
!kgtk query -i $small_graph \
     --match '(na)<-[:name]-(a)-[r:colleague]->(b)-[:name]->(nb)' \
     --where 'na.kgtk_lqstring_lang = "de" OR nb.kgtk_lqstring_lang = "de"' \
     --return 'r, na as node1, r.label, nb as node2'

id	node1	label	node2
e4	'Gary R. Rydstrom'@en	colleague	'Arnold Schwarzenegger'@de


**Example**
- it is good practice to only name pattern variables that are actually needed and leave others anonymous

In [33]:
!kgtk query -i $small_graph \
     --match '(na)<-[:name]-()-[r:colleague]->()-[:name]->(nb)' \
     --return 'r, na as node1, r.label, nb as node2'

id	node1	label	node2
e3	'Gary R. Rydstrom'@en	colleague	'Gloria S. Borders'@en
e4	'Gary R. Rydstrom'@en	colleague	'Arnold Schwarzenegger'@de


## Querying connected edges across multiple graphs

- Kypher can combine information from different graphs described in separate KGTK files
- allows one to mix and match information, or to organize data into different logical parts
- to query multiple graphs we need to specify two or more input files
- *graph variables* are used to associate pattern clauses with the graph they apply to
- for graphs to connect they need to have some node or edge IDs in common
- **Kypher vs. Cypher*: Cypher does not address multi-graph queries
- to illustrate this we use a second example graph of film data shown here:

In [34]:
!echo $small_films

/root/isi-kgtk-tutorial/tutorial-kypher_input/small-films.tsv


In [35]:
!kgtk query -i $small_films --match '()-[]->()'

id	node1	label	node2
t1	terminator2_jd	name	'Terminator 2'@en
t2	terminator2_jd	instance_of	film
t3	terminator2_jd	genre	science_fiction
t4	terminator2_jd	genre	action
t5	terminator2_jd	cast	a_schwarzenegger
t6	terminator2_jd	cast	l_hamilton
t7	terminator2_jd	crew	g_rydstrom
t8	terminator2_jd	crew	g_borders
t9	terminator2_jd	award	academy-best-sound-editing
t10	terminator2_jd	publication_date	^1991-07-03T00:00:00Z/11
t11	terminator2_jd	publication_date	^1995-07-01T00:00:00Z/11
t12	film	subclass_of	visual_artwork
t13	s_connor	name	'Sarah Connor'@en
t14	terminator	name	'Terminator'@en


**Example**
- let's query for people whose spouse is a film actor
- a graph variable followed by `:` means all clauses following it are to be applied to that graph,
  until overridden by another graph variable
- graph variables are matched to input files in a simple greedy matching scheme (see manual),
  for example, here we use a substring of each input file name
- multiple edges in the pattern are separated by commas to allow for graph switching

In [36]:
!kgtk query -i $small_graph -i $small_films \
     --match 'graph: (x)-[r:spouse]->(y), \
              films: (y)<-[:cast]-(f)-[:instance_of]->(:film)' \
     --return 'r, x, r.label, y, f as `node2;acted_in`'

id	node1	label	node2	node2;acted_in
e1	m_shriver	spouse	a_schwarzenegger	terminator2_jd


**Example**
- let's further restrict that the acting had to be done before 1985

In [37]:
!kgtk query -i $small_graph -i $small_films \
     --match 'graph: (x)-[r:spouse]->(y), \
              films: (y)<-[:cast]-(f)-[:instance_of]->(:film), \
                     (f)-[:publication_date]->(d)' \
     --where 'd < "^1995"' \
     --return 'r, x, r.label, y, f as `node2;acted_in`, d as `node2;acted_when`'

id	node1	label	node2	node2;acted_in	node2;acted_when
e1	m_shriver	spouse	a_schwarzenegger	terminator2_jd	^1991-07-03T00:00:00Z/11


## Aggregation

- similar to SQL and Cypher, Kypher supports aggregation functions such
  as `count`, `min`, `max`, `avg`, etc.
- one of the simplest and most frequently used operations is counting rows via `count`

**Example**
- count how many edges have `terminator2_jd` as their starting node

In [38]:
!kgtk query -i $small_films \
     --match '(:terminator2_jd)-[r]->()' \
     --return 'count(r) as N'

N
11


**Example**
- counts may include duplicate values (every selected edge will be counted)
- we can use `distinct` keyword as the first argument to `count` (or any aggregation function) to exclude duplicates

In [39]:
!kgtk query -i $small_films \
     --match '(:terminator2_jd)-[r]->()' \
     --return 'count(distinct r.label) as N'

N
7


**Example**
- Kypher does not have an explicit `group by` clause (different from SPARQL or SQL)
- grouping is inferred from clause type and order in the `return` statement
  and sorts rows into into groups before an aggregation operation is applied to each group
- below we group by relation label and then select the maximum `node2` for each label (lexicographically)

In [40]:
!kgtk query -i $small_films \
     --match '(x)-[r]->(y)' \
     --return 'r.label, max(y) as node2, x, r'

label	node2	node1	id
award	academy-best-sound-editing	terminator2_jd	t9
cast	l_hamilton	terminator2_jd	t6
crew	g_rydstrom	terminator2_jd	t7
genre	science_fiction	terminator2_jd	t3
instance_of	film	terminator2_jd	t2
name	'Terminator'@en	terminator	t14
publication_date	^1995-07-01T00:00:00Z/11	terminator2_jd	t11
subclass_of	visual_artwork	film	t12


**Mildly tricky**: the query applied the `max` function to groups of
result rows where `r.label` had the same value.  But for this to
work we had to move the other output variables `x` and `r` to the
end, otherwise they would have been the grouping criterion (that's
the drawback of implicit grouping).

## Optional match

- real-world KGs often only have partial coverage of certain relations
- for example, not every node might have a name or label
- Kypher's optional match patterns can be used to hande incomplete information,
  since they are allowed to fail and will generate NULL values for such cases
- each Kypher query must have exactly one strict `--match` clause and
  can have zero or more optional match clauses introduced by `--opt`
- **Kypher vs. Cypher**: this is more than Cypher which can have any
  number of strict and/or optional patterns in any order
- each strict and optional match clause can have its own `--where` clause
- below we sometimes use a third graph of edge qualifiers shown here:

In [41]:
!echo $small_quals

/root/isi-kgtk-tutorial/tutorial-kypher_input/small-quals.tsv


In [42]:
!kgtk query -i $small_quals

id	node1	label	node2
q1	t5	role	terminator
q2	t6	role	s_connor
q3	t9	point_in_time	^1992-03-30T00:00:00Z/11
q4 	t9	winner	g_rydstrom
q5 	t9	winner	g_borders
q6 	t10	location	united_states
q8 	t11	location	russia
q9	e1	start	^1986-04-26T08:00:00Z/11
q10	e1	end	^2011-07-01T10:00:00Z/11


**Example**
- start with a strict query to find people associated with a movie and
  any awards they may have won

In [43]:
!kgtk query -i $small_graph -i $small_films -i $small_quals \
     --match  'graph: (p)-[:name]->(), \
               films: (f)-[:instance_of]->(:film), (f)-[]->(p), (f)-[ra:award]->(a), \
               quals: (ra)-[:winner]->(p)' \
     --return 'p as person, f as film, a as award'

person	film	award
g_rydstrom	terminator2_jd	academy-best-sound-editing
g_borders	terminator2_jd	academy-best-sound-editing


- result only lists some of the people, since not all participants won awards
- this makes us miss some potentially useful data
- to make sure we retrieve all people associated with a movie and optionally
  any awards they may have received, we can use this query:

In [44]:
!kgtk query -i $small_graph -i $small_films -i $small_quals \
     --match  'graph: (p)-[:name]->(), \
               films: (f)-[:instance_of]->(:film), (f)-[]->(p)' \
     --opt    'films: (f)-[ra:award]->(a), \
               quals: (ra)-[:winner]->(p)' \
     --return 'p as person, f as film, a as award'

person	film	award
a_schwarzenegger	terminator2_jd	
l_hamilton	terminator2_jd	
g_rydstrom	terminator2_jd	academy-best-sound-editing
g_borders	terminator2_jd	academy-best-sound-editing


- now we get all relevant people and missing awards are simply empty (or NULL)
- note how edge qualifiers associate information to an edge ID (e.g., the winner of an award)
- optional patterns are either fully satisfied for a set of bindings, or not at all,
  there are no partial matches

**Example**
- multiple independent optional clauses to also get spouses and to filter awards
  after 1990

In [45]:
!kgtk query -i $small_graph -i $small_films -i $small_quals \
     --match  'graph: (p)-[:name]->(), \
               films: (f)-[:instance_of]->(:film), (f)-[]->(p)' \
     --opt    'films: (f)-[ra:award]->(a), \
               quals: (ra)-[:winner]->(p), (ra)-[:point_in_time]->(ad)' \
     --where  'ad >= 1990' \
     --opt    'graph: (s)-[:spouse]->(p)' \
     --return 'p as person, f as film, a as award, s as spouse'

person	film	award	spouse
a_schwarzenegger	terminator2_jd		m_shriver
l_hamilton	terminator2_jd		
g_rydstrom	terminator2_jd	academy-best-sound-editing	
g_borders	terminator2_jd	academy-best-sound-editing	


**Example**
- compare that to the SQL produced

In [46]:
!kgtk --debug query -i $small_graph -i $small_films -i $small_quals \
     --match  'graph: (p)-[:name]->(), \
               films: (f)-[:instance_of]->(:film), (f)-[]->(p)' \
     --opt    'films: (f)-[ra:award]->(a), \
               quals: (ra)-[:winner]->(p), (ra)-[:point_in_time]->(ad)' \
     --where  'ad >= 1990' \
     --opt    'graph: (s)-[:spouse]->(p)' \
     --return 'p as person, f as film, a as award, s as spouse'

[2022-10-04 22:41:22 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."node1" "_aLias.person", graph_2_c2."node1" "_aLias.film", graph_2_c4."node2" "_aLias.award", graph_1_c7."node1" "_aLias.spouse"
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_2 AS graph_2_c2, graph_2 AS graph_2_c3
     ON graph_1_c1."node1" = graph_2_c3."node2"
        AND graph_2_c2."node1" = graph_2_c3."node1"
        AND graph_1_c1."label" = ?
        AND graph_2_c2."label" = ?
        AND graph_2_c2."node2" = ?
     LEFT JOIN (graph_2 AS graph_2_c4
         INNER JOIN graph_3 AS graph_3_c5, graph_3 AS graph_3_c6
         ON graph_2_c4."id" = graph_3_c5."node1"
            AND graph_2_c4."id" = graph_3_c6."node1"
            AND graph_2_c4."label" = ?
            AND graph_3_c5."label" = ?
            AND graph_3_c6."label" = ?
            AND (graph_3_c6."node2" >= ?))
     ON graph_1_c1."node1" = graph_3_c5."node2"
        AND graph_2_c2."node1" = graph_2_c4."nod

- see the manual for more examples
- e.g., how to do `NOT EXISTS` with an optional clause

## Full-text search

- Kypher supports efficient full-text search over large graph data via SQLite's FTS5 module
- matching uses specialized indexes which can be created easily and efficiently

**Example**
- we define a text index on the `node2` column of `GRAPH` and then use `textmatch` to match against `node2` values
- multiple indexes can be defined on the same graph column, we can provide names to disambiguate
- **minor caveat**: use of an older Python version requires us to provide extra index definition options,
  since the default `trigram` tokenizer is currently only supported in Python 3.9 or later
- match scores are negative with the best being the smallest (most negative) 

In [47]:
!rm -f $KGTK_GRAPH_CACHE  # bug workaround

In [48]:
!kgtk query -i $small_graph --idx auto text:node2//name=myidx//tokenize=ascii//prefix=3 \
     --match '(x)-[r]->(y)' \
     --where 'textmatch(y, "sch*")' \
     --return 'x, r.label, y, matchscore(y) as score' \
     --order 'score'

node1	label	node2	score
m_shriver	spouse	a_schwarzenegger	-0.6991202559002525
g_rydstrom	colleague	a_schwarzenegger	-0.6991202559002525
a_schwarzenegger	name	'Arnold Schwarzenegger'@de	-0.5994217687032093


**Example**
- `textmatch` patterns use a phrase-based language that allows
  multi-word phrases, Boolean expressions, multi-column expressions, suffix patterns, etc. (see manual)
- here is an example of a Boolean expression
- note that the index only needs to be specified the first time around, repeating it is a no-op

In [49]:
!kgtk query -i $small_graph \
     --match '(x)-[r]->(y)' \
     --where 'textmatch(y, "sch* OR linda")' \
     --return 'x, r.label, y, matchscore(y) as score' \
     --order 'score'

node1	label	node2	score
l_hamilton	name	'Linda Hamilton'@en	-1.6796313036328494
m_shriver	spouse	a_schwarzenegger	-0.6991202559002525
g_rydstrom	colleague	a_schwarzenegger	-0.6991202559002525
a_schwarzenegger	name	'Arnold Schwarzenegger'@de	-0.5994217687032093


The `trigram` tokenizer (available with Python 3.9) also supports
case-insensitive SQL `LIKE` patterns and case-sensitive `GLOB`
patterns (see manual).

**Example**
- text indexes can be qualified with their name to select between multiple options

In [50]:
!kgtk query -i $small_graph \
     --match '(x)-[r]->(y)' \
     --where 'textmatch(myidx.y, "schw*")' \
     --return 'x, r.label, y, matchscore(myidx.y) as score' \
     --order 'score'

node1	label	node2	score
m_shriver	spouse	a_schwarzenegger	-0.6991202559002525
g_rydstrom	colleague	a_schwarzenegger	-0.6991202559002525
a_schwarzenegger	name	'Arnold Schwarzenegger'@de	-0.5994217687032093


**Example**
- a text index may index more than one column
- then the text match expression can use column-specific filters

In [51]:
!kgtk query -i $small_graph --idx auto text:node1,node2//name=multi//tokenize=ascii//prefix=3 \
     --match '(x)-[r]->(y)' \
     --where 'textmatch(multi.r, "node1: g* AND node2 : borders")' \
     --return 'x, r.label, y, matchscore(multi.r) as score' \
     --order 'score'

node1	label	node2	score
g_rydstrom	colleague	g_borders	-1.391983808592556
g_borders	name	'Gloria S. Borders'@en	-1.1761930172126576


In [52]:
!kgtk query -i $small_graph \
     --match '(x)-[r]->(y)' \
     --where 'textmatch(multi.r, "node1: g* NOT node2 : borders")' \
     --return 'x, r.label, y, matchscore(multi.r) as score' \
     --order 'score'

node1	label	node2	score
g_rydstrom	colleague	a_schwarzenegger	-0.2149880837001077
g_rydstrom	name	'Gary R. Rydstrom'@en	-0.18165978747100012


## Input and output specification options

```
  -i INPUT_FILE [INPUT_FILE ...], --input-files INPUT_FILE [INPUT_FILE ...]
                        One or more input files to query (maybe compressed).
                        (Required, use '-' for stdin.)
  --as NAME             alias name to be used for preceding input
  --comment COMMENT     comment string to store for the preceding input
                        (displayed by --show-cache)
  --no-header           do not generate a header row with column names
  -o OUTPUT, --out OUTPUT
                        output file to write to, if `-' (the default) output
                        goes to stdout. Files with extensions .gz, .bz2 or .xz
                        will be appropriately compressed.
```

**Example**

In [53]:
!kgtk query -i $small_graph -i $small_films --as works --comment 'Film information' --no-header \
     --match 'graph: (x)-[]->(y), works: (f)-[r]->(x)' \
     --return 'r, f, r.label, x' \
     -o $TEMP/example-query.tsv.gz

## Graph cache

- when input files are queries they are first imported into SQLite database tables
- this is very fast but can be noticable for larger data
- import of WD claims file with 1B edges (~16 GB compressed) takes 20 minutes on laptop
- resulting database is cached into a *graph cache* to amortize this over multiple queries
- indexes built to speed up queries or text search are also cached
- if data change is detected, data will be re-imported and re-indexed automatically

- location of the cache file can be controlled with `--graph-cache FILE` or the `KGTK_GRAPH_CACHE`
  environment variable
- otherwise a default location in `/tmp` will be used

- cache is a *true cache*, that is it is safe to delete it and it will be rebuilt automatically
  (as long as the relevant data files are still available)
- however, we increasingly find cases where the cache is explicitly managed, for example,
  to support the browser or to ship large data to somebody else
- cache files can become very large and should be on an SSD drive for best performance

**Example**
- display the current content of the cache with `--show-cache`

In [54]:
!kgtk query --show-cache

Graph Cache:
DB file: /tmp/projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
  size:  72.00 KB   	free:  0 Bytes   	modified:  2022-10-04 22:41:29

KGTK File Information:
/root/isi-kgtk-tutorial/tutorial-kypher_input/small-graph.tsv:
  size:  382 Bytes   	modified:  2022-10-04 22:40:36   	graph:  graph_1
works:
  size:  590 Bytes   	modified:  2022-10-04 22:40:37   	graph:  graph_2
  comment:  Film information

Graph Table Information:
graph_1:
  size:  40.00 KB   	created:  2022-10-04 22:41:24
  header:  ['id', 'node1', 'label', 'node2']
graph_2:
  size:  12.00 KB   	created:  2022-10-04 22:41:29
  header:  ['id', 'node1', 'label', 'node2']


## Defining and using custom functions

- when provided built-in functions are not enough, custom functions can be executed via Python

**Example**
- perform some random modifications on the `GRAPH` data
- note that values returned by `pyeval` and `pycall` must be simple literals

In [55]:
!kgtk query -i $small_graph --import 'uuid, math as m' \
     --match '(x)-[r:name]->(y)' \
     --where 'kgtk_lqstring(y)' \
     --return 'y as name, \
               pyeval(printf($FMT, y)) as swapname, \
               pycall("m.fmod", length(y), 2) as isodd, \
               pycall("uuid.uuid4") as uuid' \
     --para FMT='"%s".swapcase()'

name	swapname	isodd	uuid
'Arnold Schwarzenegger'@de	'aRNOLD sCHWARZENEGGER'@DE	0.0	1f6ce5fd-427d-453a-95be-ca7d8a4b576b
'Linda Hamilton'@en	'lINDA hAMILTON'@EN	1.0	116b047c-a63e-4ebf-ba0e-a3414d925566
'Gary R. Rydstrom'@en	'gARY r. rYDSTROM'@EN	1.0	bea02bb7-427b-4c3e-81db-d0adb2db5c6c
'Gloria S. Borders'@en	'gLORIA s. bORDERS'@EN	0.0	9eb268ea-de00-4183-92fb-459e440d5faf
'Maria Shriver'@en	'mARIA sHRIVER'@EN	0.0	1b85f1fc-0a92-4930-89d4-2d459ce4c28a


## Debugging

**Example**
- `query` accepts the `--debug` and `--expert` options to show additional output
- this shows query translation, data import and indexing, etc.
- here we can see how built-in functions are called directly in SQL:

In [56]:
!rm -f $KGTK_GRAPH_CACHE

!kgtk --debug query -i $small_graph \
     --match '(p)-[r:name]->(n)' \
     --where 'n.kgtk_lqstring_lang = "de"'

[2022-10-04 22:41:32 sqlstore]: IMPORT graph via csv.reader into table graph_1 from /root/isi-kgtk-tutorial/tutorial-kypher_input/small-graph.tsv ...
[2022-10-04 22:41:32 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = ?
        AND (kgtk_lqstring_lang(graph_1_c1."node2") = ?)
  PARAS: ['name', 'de']
---------------------------------------------
[2022-10-04 22:41:32 sqlstore]: CREATE INDEX "graph_1_label_idx" ON "graph_1" ("label")
[2022-10-04 22:41:32 sqlstore]: ANALYZE "graph_1_label_idx"
id	node1	label	node2
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de


- run it again to show how imported data and indexes are reused

In [57]:
!kgtk --debug query -i $small_graph \
     --match '(p)-[r:name]->(n)' \
     --where 'n.kgtk_lqstring_lang = "de"'

[2022-10-04 22:41:33 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = ?
        AND (kgtk_lqstring_lang(graph_1_c1."node2") = ?)
  PARAS: ['name', 'de']
---------------------------------------------
id	node1	label	node2
e5	a_schwarzenegger	name	'Arnold Schwarzenegger'@de


## Querying based on edge qualifiers

- one of the motivations for developing Kypher was the "Wikidata time machine use case"
- find all statements with temporal annotations after a certain time and remove them
  to only retain facts known before that time
- in Wikidata temporal annotations are attached via edge qualifiers
- in KGTK those are represented via edges linked to another edge's ID

**Example**
1. look for base edges in the `GRAPH` graph
2. link to qualifiers in the `QUALS` graph via edge id `r`
3. restrict the qualifiers based on edge labels
   `ql` that are listed in the `PROPS` graph
4. restrict to edges that have a time with year of at most 2000
5. output the qualifying base edges with their temporal annotations

In [58]:
!kgtk query -i $small_quals

id	node1	label	node2
q1	t5	role	terminator
q2	t6	role	s_connor
q3	t9	point_in_time	^1992-03-30T00:00:00Z/11
q4 	t9	winner	g_rydstrom
q5 	t9	winner	g_borders
q6 	t10	location	united_states
q8 	t11	location	russia
q9	e1	start	^1986-04-26T08:00:00Z/11
q10	e1	end	^2011-07-01T10:00:00Z/11


In [59]:
!kgtk query -i $small_props

id	node1	label	node2
p11	start	member	set1
p12	end	member	set1
p12	point_in_time	member	set1


- the crucial part of the query below is how we use the `id` of the base edge `r` as the `node1` of the qualifier edge `q` whose label `ql` has to be one of the properties listed in `PROPS`
- we also need to use Kypher property syntax to introduce the match variable `ql`

In [60]:
!kgtk query -i $small_graph -i $small_quals -i $small_props  \
     --match "graph: (x)-[r]->(y),  \
              quals: (r)-[q {label: ql}]->(time), \
              props: (ql)-[]->()" \
     --where "time.kgtk_date_year <= 2000" \
     --return "r as id, x, r.label, y, ql as trel, time as time"

id	node1	label	node2	trel	time
e1	m_shriver	spouse	a_schwarzenegger	start	^1986-04-26T08:00:00Z/11


## Further reading

- [Kypher manual](https://github.com/usc-isi-i2/kgtk/blob/dev/docs/transform/query.md)
  - [Kypher vs. Cypher and unfinished features](https://github.com/usc-isi-i2/kgtk/blob/dev/docs/transform/query.md#differences-to-cypher)
  - [Kypher API](https://github.com/usc-isi-i2/kgtk/blob/dev/kgtk/kypher/api.py)
- [KGTK manual](https://kgtk.readthedocs.io/en/dev/)
- [KGTK data model](https://kgtk.readthedocs.io/en/dev/data_model/)
- [KGTK GitHub site](https://github.com/usc-isi-i2/kgtk)
- [KGTK Tutorial use cases](https://github.com/usc-isi-i2/kgtk-notebooks/tree/main/tutorial)
- [KGTK use cases](https://github.com/usc-isi-i2/kgtk/tree/dev/use-cases)
- [openCypher](https://www.opencypher.org/)
- [Cypher](https://neo4j.com/developer/cypher/)